In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import RMSprop

In [ ]:
# Set the paths for the training and testing datasets
train_data_dir = 'cats-or-dogs/train_set'
test_data_dir = 'cats-or-dogs/test_set'
cats_dir = '/cats'
dogs_dir = '/dogs'

# Define the image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 20
train_samples = 200
test_samples = 40

In [ ]:
train_cat_files = os.listdir(train_data_dir + cats_dir)
train_dog_files = os.listdir(train_data_dir + dogs_dir)
test_cat_files = os.listdir(test_data_dir + cats_dir)
test_dog_files = os.listdir(test_data_dir + dogs_dir)

In [ ]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')


validation_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False)


In [ ]:
# Use VGG16 as a pre-trained model for embeddings
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_width, img_height, 3))



In [ ]:
# Training a classifier
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

conv_base.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=2e-5),
              metrics=['acc'])



In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch= train_samples // batch_size,
      epochs=30,
      validation_data=validation_generator)

In [ ]:
# Testing accuracy
test_loss, test_acc = model.evaluate(validation_generator, steps=test_samples // batch_size)
print('Test accuracy:', test_acc)

In [ ]:
# Show confusion matrix
test_labels = validation_generator.classes
test_predictions = model.predict(validation_generator).flatten() > 0.5
conf_matrix = confusion_matrix(test_labels, test_predictions)
print('Confusion Matrix:')
print(conf_matrix)

In [ ]:
labels = ["Cats", "Dogs"]
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()